In [ ]:
import os
import pandas as pd
import openai
from google.colab import files
from IPython.display import display  # Colab에서 DataFrame 출력용 / For displaying DataFrames in Colab

# ===============================
# 🔹 Step 1: API 키 파일 업로드 및 확인
# 🔹 Step 1: Upload and verify the OpenAI API key file
# ===============================
api_key_path = "/content/가영 API for 논문.txt"  # File path for API key (should be in /content/ in Colab)

if not os.path.exists(api_key_path):
    print("API 키 파일을 업로드하세요.")  # Prompt user to upload the key file
    uploaded = files.upload()
    if "가영 API for 논문.txt" not in uploaded:
        raise FileNotFoundError("API 키 파일이 업로드되지 않았습니다.")  # Raise error if not found

with open(api_key_path, "r") as file:
    api_key = file.read().strip()  # Read the API key

# 🔹 Create OpenAI API client
client = openai.OpenAI(api_key=api_key)

# ===============================
# 🔹 Step 2: 엑셀 파일 업로드 및 확인
# 🔹 Step 2: Upload and validate Excel file
# ===============================
print("엑셀 파일을 업로드하세요.")  # Ask user to upload Excel file
uploaded = files.upload()
uploaded_files = list(uploaded.keys())

if not uploaded_files:
    raise FileNotFoundError("엑셀 파일이 업로드되지 않았습니다.")  # No file uploaded

file_path = uploaded_files[0]  # Use the first uploaded file
output_file = "/content/gpt4o_hydrogen_economy_article_classifier.xlsx"  # English output file name

print(f"사용할 파일: {file_path}")  # Show which file will be used

# ===============================
# 🔹 Step 3: 엑셀 파일 읽기
# 🔹 Step 3: Read Excel file
# ===============================
df = pd.read_excel(file_path)

# Check if 'text' column exists
if 'text' not in df.columns:
    raise ValueError("엑셀 파일에 'text' 컬럼이 존재하지 않습니다.")  # Must include 'text' column

# ===============================
# 🔹 Step 4: GPT-4o를 사용한 관련성 분석
# 🔹 Step 4: Relevance tagging using GPT-4o
# ===============================
keywords = ['수소', '연료전지', '수소차', '수소경제', '수소에너지', '수소 저장', '수소 생산', '그린수소', '블루수소']

def check_relevance_gpt(text):
    """
    🔍 Classifies the text as:
    - 'Y': clearly related to the hydrogen economy
    - 'N': clearly unrelated
    - 'V': uncertain, needs manual review
    """
    prompt = f"""
    아래 기사 내용이 '수소 경제'와 직접적인 관련이 있는지 판단해줘.
    # Determine whether the following news article is directly related to the hydrogen economy.

    ✅ 'Y'로 태깅해야 하는 경우:
    # ✅ Tag as 'Y' (Yes) if any of the following apply:
    - 연료전지, 수소차, 수소 생산·저장·운송, 수소 기반 발전 등 수소 경제 핵심 산업 내용이 포함된 경우
    # - Includes content related to key hydrogen economy industries: fuel cells, hydrogen cars, production, storage, transportation, power generation

    - 기업, 연구기관, 정부 정책, 수소 기술 개발 등 경제·산업적으로 의미 있는 내용
    # - Includes meaningful economic/industrial content such as corporate activities, R&D, or government policy on hydrogen

    ❌ 'N'으로 태깅해야 하는 경우:
    # ❌ Tag as 'N' (No) if any of the following apply:
    - 삼중수소(T₂), 핵융합, 방사선 연구, 의료용 동위원소 등 수소 경제와 무관한 경우
    # - Content is about tritium (T₂), fusion, medical isotopes, or radiation research unrelated to hydrogen economy

    - '수소'라는 단어가 포함되었지만 수소 경제와 직접적 관련이 없는 경우
    # - "Hydrogen" appears, but there is no meaningful connection to the hydrogen economy

    - 광고, 금융 공시, 단순 기술 소개 (수소 연료와 무관한 내용)
    # - Ads, financial disclosures, or general technologies not involving hydrogen fuel

    ⚠️ 'V' (검토 필요)로 태깅해야 하는 경우:
    # ⚠️ Tag as 'V' (Unclear – needs review) if any of the following apply:
    - 연관성이 있어 보이지만 불확실하거나 모호한 경우
    # - Appears related but unclear or contextually ambiguous

    - 예: "수소 활용 연구 진행 중", "수소 관련 정부 지원 논의" 등
    # - Example: "Hydrogen utilization study underway", "Government support for hydrogen under discussion"

    - 명확한 판단이 어려운 기사는 'V'로 태깅
    # - Tag as 'V' if classification is ambiguous

    반드시 아래 중 하나로만 답해주세요:
    # Please respond with only one of the following:
      - "Y" / "N" / "V"

    기사 내용: "{text}"
    # Article content to evaluate
    """

    response = client.chat.completions.create(
        model="gpt-4o",  # Using GPT-4o model for classification
        messages=[{"role": "system", "content": "You are an AI assistant trained to classify text relevance."},
                  {"role": "user", "content": prompt}],
        max_tokens=10
    )

    result = response.choices[0].message.content.strip()

    if "Y" in result:
        return "Y"
    elif "V" in result:
        return "V"
    else:
        return "N"

# Apply GPT classifier to all rows in the 'text' column
df['관련 여부'] = df['text'].astype(str).apply(check_relevance_gpt)

# ===============================
# 🔹 Step 5: 결과 저장 및 다운로드
# 🔹 Step 5: Save results and download as Excel
# ===============================
df.to_excel(output_file, index=False)
files.download(output_file)

# ===============================
# 🔹 Step 6: 'V'로 태깅된 기사 샘플 출력
# 🔹 Step 6: Display up to 10 articles tagged as 'V' (uncertain)
# ===============================
num_v_samples = min(10, len(df[df['관련 여부'] == 'V']))
if num_v_samples > 0:
    sample_v_check = df[df['관련 여부'] == 'V'].sample(n=num_v_samples, random_state=42)
    display(sample_v_check)
else:
    print("✅ 'V'로 태깅된 데이터가 없어서 샘플링을 건너뜁니다.")  # No uncertain articles found

print(f"✅ 처리 완료: {output_file}")  # Task completed


엑셀 파일을 업로드하세요.


Saving total data_articles_combined_edited_split3..xlsx to total data_articles_combined_edited_split3..xlsx
사용할 파일: total data_articles_combined_edited_split3..xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 'M'으로 태깅된 데이터가 없어서 샘플링을 건너뜁니다.
처리 완료: /content/output.xlsx
